### Cloud Watch Logs (CWL)
##### Cloud Watch Log Management

In [ ]:
import datetime
import botoplus.organization as _org

display_denied = False

accounts = _org.accounts()

daily = 0
total = 0
cwlogs = 0

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    sessions = _org.sessions(account['awsaccount'])
    ec2 = sessions.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        display_region = True
        try:
            logs = sessions.client('logs', region_name = region['RegionName'])
            paginator = logs.get_paginator('describe_log_groups')
            pages = paginator.paginate()
            for page in pages:
                if len(page['logGroups']) > 0:
                    if display_region == True:
                        print(' - '+region['RegionName'])
                        display_region = False
                    for group in page['logGroups']:
                        cwlogs += 1
                        age = (datetime.datetime.now() - datetime.datetime.fromtimestamp(group['creationTime']/1000)).days
                        try:
                            group['retentionInDays'] = group['retentionInDays']
                        except:
                            group['retentionInDays'] = 'Infinity'
                        group['storedBytes'] = round(group['storedBytes']/1024/1024/1024,2)
                        total += group['storedBytes']
                        if group['logGroupClass'] == 'STANDARD':
                            cost = 0.50
                        else:
                            cost = 0.25
                        if group['retentionInDays'] == 'Infinity':
                            price = round(((group['storedBytes']/age) * cost),2)
                        else:
                            price = round(((group['storedBytes']/group['retentionInDays']) * cost),2)
                        daily += price
                        print('   '+group['logGroupName']+' Age: '+str(age)+' Retention: '+str(group['retentionInDays'])+' GBs: '+str(group['storedBytes'])+' Estimated Daily: $'+str(price))
        except Exception as e:
            pass

print('CloudWatch Log Groups: '+str(cwlogs))
print('Estimated Daily: $'+str(daily))
print('Storage Cost: $'+str(round((round(total,2)*0.03),2)))
print('Total: '+str(round(total,2))+' GBs')

##### Cloud Watch Logs - No Expiration

In [ ]:
import botoplus.organization as _org

display_denied = False

accounts = _org.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    sessions = _org.sessions(account['awsaccount'])
    ec2 = sessions.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        display_region = True
        try:
            logs = sessions.client('logs', region_name = region['RegionName'])
            paginator = logs.get_paginator('describe_log_groups')
            pages = paginator.paginate()
            for page in pages:
                if len(page['logGroups']) > 0:
                    if display_region == True:
                        print(' - '+region['RegionName'])
                        display_region = False
                    for group in page['logGroups']:
                        try:
                            group['retentionInDays'] = group['retentionInDays']
                        except:
                            group['retentionInDays'] = 'Infinity'
                            print('   '+group['logGroupName'])
        except:
            if display_denied == True:
                print(' - '+region['RegionName']+' DENIED')
            pass

##### Cloud Watch Logs - Set Expiration

In [ ]:
import botoplus.organization as _org

display_denied = False

accounts = _org.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    sessions = _org.sessions(account['awsaccount'])
    ec2 = sessions.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        display_region = True
        try:
            logs = sessions.client('logs', region_name = region['RegionName'])
            paginator = logs.get_paginator('describe_log_groups')
            pages = paginator.paginate()
            for page in pages:
                if len(page['logGroups']) > 0:
                    if display_region == True:
                        print(' - '+region['RegionName'])
                        display_region = False
                    for group in page['logGroups']:
                        try:
                            group['retentionInDays'] = group['retentionInDays']
                        except:
                            group['retentionInDays'] = 'Infinity'
                            response = logs.put_retention_policy(
                                logGroupName = group['logGroupName'],
                                retentionInDays = 30
                            )
                            print('   '+str(response['ResponseMetadata']['HTTPStatusCode'])+' '+group['logGroupName'])
        except:
            if display_denied == True:
                print(' - '+region['RegionName']+' DENIED')
            pass

##### Cloud Watch Logs - No Storage

In [ ]:
import botoplus.organization as _org

display_denied = False

accounts = _org.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    sessions = _org.sessions(account['awsaccount'])
    ec2 = sessions.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        display_region = True
        try:
            logs = sessions.client('logs', region_name = region['RegionName'])
            paginator = logs.get_paginator('describe_log_groups')
            pages = paginator.paginate()
            for page in pages:
                if len(page['logGroups']) > 0:
                    if display_region == True:
                        print(' - '+region['RegionName'])
                        display_region = False
                    for group in page['logGroups']:
                        if group['storedBytes'] == 0:
                            print('   '+group['logGroupName'])
        except:
            if display_denied == True:
                print(' - '+region['RegionName']+' DENIED')
            pass

##### Cloud Watch Logs - Clean Up

In [ ]:
import botoplus.organization as _org

display_denied = False

accounts = _org.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    sessions = _org.sessions(account['awsaccount'])
    ec2 = sessions.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        display_region = True
        try:
            logs = sessions.client('logs', region_name = region['RegionName'])
            paginator = logs.get_paginator('describe_log_groups')
            pages = paginator.paginate()
            for page in pages:
                if len(page['logGroups']) > 0:
                    if display_region == True:
                        print(' - '+region['RegionName'])
                        display_region = False
                    for group in page['logGroups']:
                        if group['storedBytes'] == 0:
                            response = logs.delete_log_group(
                                logGroupName = group['logGroupName']
                            )
                            print('   '+str(response['ResponseMetadata']['HTTPStatusCode'])+' '+group['logGroupName'])
        except:
            if display_denied == True:
                print(' - '+region['RegionName']+' DENIED')
            pass